In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# CrossProductComp

`CrossProductComp` performs a cross product between two 3-vector inputs.  It may be vectorized to provide the result at one or more points simultaneously.

$$
    c_i = \bar{a}_i \times \bar{b}_i
$$

The first dimension of the inputs holds the vectorized dimension.
The default `vec_size` is 1, providing the cross product of $a$ and $b$ at a single
point.  The lengths of $a$ and $b$ at each point must be 3.

The shape of $a$ and $b$ will always be `(vec_size, 3)`, but the connection rules
of OpenMDAO allow the incoming connection to have shape `(3,)` when `vec_size` is 1, since
the storage order of the underlying data is the same.  The output vector `c` of
CrossProductComp will always have shape `(vec_size, 3)`.

## CrossProductComp Options

Options for CrossProductComp allow the user to rename the input variables $a$ and $b$ and the output $c$, as well as specifying their units.

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.components.cross_product_comp.CrossProductComp")

## CrossProductComp Constructor

The call signature for the `CrossProductComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.cross_product_comp.CrossProductComp.__init__
        :noindex:
```

## CrossProductComp Usage

There are often situations when numerous products need to be computed, essentially in parallel.
You can reduce the number of components required by having one `CrossProductComp` perform multiple operations.
This is also convenient when the different operations have common inputs.

The `add_product` method is used to create additional products after instantiation.

```{eval-rst}
    .. automethod:: openmdao.components.cross_product_comp.CrossProductComp.add_product
       :noindex:
```

## CrossProductComp Example

In the following example CrossProductComp is used to compute torque as the
cross product of force ($F$) and radius ($r$) at 100 points simultaneously.
Note the use of `a_name`, `b_name`, and `c_name` to assign names to the inputs and outputs.
Units are assigned using `a_units`, `b_units`, and `c_units`.
Note that no internal checks are performed to ensure that `c_units` are consistent
with `a_units` and `b_units`.


In [ ]:
import numpy as np
import openmdao.api as om

n = 24

p = om.Problem()

p.model.add_subsystem(name='cross_prod_comp',
                      subsys=om.CrossProductComp(vec_size=n,
                                                 a_name='r', b_name='F', c_name='torque',
                                                 a_units='m', b_units='N', c_units='N*m'),
                      promotes_inputs=['r', 'F'])

p.setup()

p.set_val('r', np.random.rand(n, 3))
p.set_val('F', np.random.rand(n, 3))

p.run_model()

# Check the output in units of ft*lbf to ensure that our units work as expected.
expected = []
for i in range(n):
    a_i = p.get_val('r')[i, :]
    b_i = p.get_val('F')[i, :]
    expected.append(np.cross(a_i, b_i) * 0.73756215)

    actual_i = p.get_val('cross_prod_comp.torque', units='ft*lbf')[i]
    rel_error = np.abs(expected[i] - actual_i)/actual_i
    assert np.all(rel_error < 1e-8), f"Relative error: {rel_error}"

print(p.get_val('cross_prod_comp.torque', units='ft*lbf'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('cross_prod_comp.torque', units='ft*lbf'), np.array(expected), tolerance=1e-8)

## DotProductComp Example with Multiple Products

When defining multiple products:

- An input name in one call to `add_product` may not be an output name in another call, and vice-versa.
- The units and shape of variables used across multiple products must be the same in each one.

In [ ]:
n = 24

p = om.Problem()

cpc = om.CrossProductComp(vec_size=n,
                          a_name='r', b_name='F', c_name='torque',
                          a_units='m', b_units='N', c_units='N*m')

cpc.add_product(vec_size=n,
                a_name='r', b_name='p', c_name='L',
                a_units='m', b_units='kg*m/s', c_units='kg*m**2/s')

p.model.add_subsystem(name='cross_prod_comp', subsys=cpc,
                      promotes_inputs=['r', 'F', 'p'])

p.setup()

p.set_val('r', np.random.rand(n, 3))
p.set_val('F', np.random.rand(n, 3))
p.set_val('p', np.random.rand(n, 3))

p.run_model()

# Check the output.
expected_T = []
expected_L = []
for i in range(n):
    a_i = p.get_val('r')[i, :]
    b_i = p.get_val('F')[i, :]
    expected_T.append(np.cross(a_i, b_i))

    actual_i = p.get_val('cross_prod_comp.torque')[i]
    rel_error = np.abs(expected_T[i] - actual_i)/actual_i
    assert np.all(rel_error < 1e-8), f"Relative error: {rel_error}"

    b_i = p.get_val('p')[i, :]
    expected_L.append(np.cross(a_i, b_i))

    actual_i = p.get_val('cross_prod_comp.L')[i]
    rel_error = np.abs(expected_L[i] - actual_i)/actual_i
    assert np.all(rel_error < 1e-8), f"Relative error: {rel_error}"

print(p.get_val('cross_prod_comp.torque'))

In [ ]:
print(p.get_val('cross_prod_comp.L'))

In [ ]:
assert_near_equal(p.get_val('cross_prod_comp.torque'), np.array(expected_T), tolerance=1e-8)
assert_near_equal(p.get_val('cross_prod_comp.L'), np.array(expected_L), tolerance=1e-8)